> Dane są dwa zbiory liczb, reprezentowane przez tablice rozmiarów $ m $ i $ n $, gdzie $ m $ jest znacznie mniejsze od $ n $. Zaproponuj algorytm, który sprawdzi, czy zbiory są rozłączne.

###### UWAGA:
Ze względu na brak informacji o rozkładzie oraz zakresie liczb, nie możemy wykorzystać Bucket Sorta ani Counting Sorta. Ponieważ liczby mogą być rzeczywiste, ale niekoniecznie całkowite (brak informacji o tym, czy są one całkowite), odpada również Radix Sort. Można by się ewentualnie pokusić o Bucket Sorta z Bucket Sortem do sortowania wiaderek, ale ma on bardzo dużą złożoność pamięciową, w przypadku niekorzystnych danych. Mimo to, zazwyczaj jest on 2-3 razy szybszy od Quick Sorta. Niestety trudno uzasadnić, dlaczego akuret się wybrało taki algorytm, więc lepiej wziąć Quick Sorta, jeżeli w zadaniu nie ma powiedzianego nic, że algorytm ma działać w czasie liniowym, być stabilny lub działać w miejscu, itp.

# I Sposób (najlepszy):

### Omówienie algorytmu

Aby sprawdzić, czy zbiory są rozłączne, wystarczy sprawdzić, czy mają one część wspólną, a więc, czy choćby jeden element z jednego ze zbiorów (nieważne którego) znajduje się w drugim ze zbiorów. Najłatwiej tego dokonać, poprzez posortowanie mniejszego ze zbiorów (ponieważ sortowanie zawsze zabiera najwięcej czasu, zazwyczaj lepiej sortować mniejsze zbiory danych), a następnie przechodzić liniowo przez NIEPOSORTOWANĄ tablicę (tę większą, odpowiadającą większemu zbiorowi) i sprawdzać wyszukiwaniem binarnym w POSORTOWANEJ (mniejszej) tablicy, czy znajduje się w niej odpowiedni element.

##### Wyjaśnienie, dlaczego sortujemy mniejszą tablicę

Zastanówmy się, którą z tablic jest lepiej posortować. Wydawałoby się, że większą, bo wtedy wyszukiwanie binarne działa znacznie szybciej (chodzi o to, że skorzystalibyśmy z wyszukiwania binarnego $ m $ razy na tablicy $ n $-elementowej, a to daje nam złożoność $ O(m \cdot log(n)) < O(n \cdot log(m)) $ (dla $ m \ll n $)). Jest to jednak złe podejście, ponieważ dokładna analiza złożoności pokazuje, że sam proces sortowania tej ($ n $-elementowej) tablicy będzie trwał znacznie dłużej niż sortowanie mniejszej ($ m $-elementowej) tablicy.

Niech: <br>
$ n $ - długość dłuższej tablicy, <br>
$ m $ - długość krótszej tablicy (tu wiemy, że $ m \ll n $), <br>
Wówczas: <br>
1. Złożoność przy sortowaniu większej tablicy: <br>
$ O(n \cdot log(n)) + O(m \cdot log(n)) = O(n \cdot log(n) + m \cdot log(n)) \approx O(n \cdot log(n)) $ <br>
Przykład dla konkretnych wartości: <br>
$ n = 2^{32} $, $ m = 2^2 $ <br>
Uwaga nieformalny zapis (zakładamy, że jest to logarytm o podstawie $ 2 $, ponieważ mamy do czynienia z wyszukiwaniem binarnym oraz sortowaniem algorytmem QuickSort, w których idea polega na podziale danych na pół): <br>
$ O(n \cdot log(n) + m \cdot log(n)) = O(2^{32} \cdot log_2{2^{32}} + 2^2 \cdot log_2{2^{32})} = O(2^{32} \cdot 32 + 2^2 \cdot 32) \approx O(2^{37}) $ <br>
2. Złożoność przy sortowaniu mniejszej tablicy: <br>
$ O(m \cdot log(m)) + O(n \cdot log(m)) = O(m \cdot log(m) + n \cdot log(m)) \approx O(n \cdot log(m)) $ <br>
Przykład dla konkretnych wartości: <br>
$ n = 2^{32} $, $ m = 2^2 $ <br>
Uwaga nieformalny zapis (zakładamy, że jet to logarytm o podstawie 2): <br>
$ O(m \cdot log(m) + n \cdot log(m)) = O(2^2 \cdot log_2{2^2} + 2^{32} \cdot log_2{2^2}) = O(2^2 \cdot 2 + 2^{32} \cdot 2) \approx O(2^{33}) $

Na podstawie powyższego przykładu widzimy, że przesortowanie mniejszej tablicy daje nam lepszy rezultat: $ O(2^{33}) < O(2^{37}) $


### Implementacja algorytmu:

In [1]:
def are_disjoint(arr1: list, arr2: list) -> bool:
    # Check which array has more elements
    if len(arr1) < len(arr2):
        shorter = arr1
        longer = arr2
    else:
        shorter = arr2
        longer = arr1
    # Sort a shorter array
    quick_sort(shorter)
    # Check using a Binary Search if a shorter array
    # has at least element from the longer array
    for val in longer:
        val_idx = binary_search_first(shorter, val)
        if val_idx >= 0:
            return False
    return True


def quick_sort(arr):
    _quick_sort(arr, 0, len(arr) - 1)
    

def _quick_sort(arr, left_idx, right_idx):
    while left_idx < right_idx:
        pivot_position = _partition(arr, left_idx, right_idx)
        
        if pivot_position - left_idx < right_idx - pivot_position:
            _quick_sort(arr, left_idx, pivot_position)
            left_idx = pivot_position + 1  # I removed a tailing recursion
        else:
            _quick_sort(arr, pivot_position + 1, right_idx)
            right_idx = pivot_position  # I removed a tailing recursion
        
        
def _partition(arr, left_idx, right_idx):
    pivot = arr[left_idx]
    
    # Partition an array into 2 subarrays of elements lower than or
    # equal to a pivot and of elements greater or equal to a pivot 
    # (in this partition algorithm pivot isn't placed on a fixed position 
    # but can be also swapped like all the remaining values)
    i = left_idx - 1
    j = right_idx + 1
    while True:
        i += 1
        while arr[i] < pivot: i += 1
            
        j -= 1
        while arr[j] > pivot: j -= 1
        
        if i < j:
            _swap(arr, i, j)
        else:
            return j  # Return a pivot position after the last swap

    
def _swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]


def binary_search_first(arr: 'sorted sequence', el: 'searched element') -> int:
    left_idx = 0
    right_idx = len(arr)-1
    
    while left_idx <= right_idx:
        mid_idx = (left_idx + right_idx) // 2
        if el > arr[mid_idx]:
            left_idx = mid_idx + 1
        else:
            right_idx = mid_idx - 1
            
    return left_idx if left_idx < len(arr) and arr[left_idx] == el else -1

###### Kilka testów

In [2]:
a = [1, 2, 6, 2, 1, 7, 3]
b = [0, 5, 5, 4, -4, 12]
print(are_disjoint(a, b))

True


In [3]:
a = [1, 2, 6, 2, 1, 7, 3]
b = [0, 5, 5, 4, -4, 12, 2]
print(are_disjoint(a, b))

False


In [4]:
import random

longer_length  = 2**16
shorter_length = 2**2

mul = 100
round_ = 3
arr1 = [round(random.random() * mul * random.choice([-1, 1]), round_) for _ in range(longer_length)]
arr2 = [round(random.random() * mul * random.choice([-1, 1]), round_) for _ in range(shorter_length)]

print(len(arr1), len(arr2))
intersection = set(arr1) & set(arr2)
expected = not bool(intersection)
result = are_disjoint(arr1, arr2)
print('Expected:', expected)
print('Result:  ', result)
print('Intersecting values:', intersection)
for val in intersection:
    print(f'Is {val} in arr1?: {val in arr1}\tIs {val} in arr2?: {val in arr2}')

65536 4
Expected: True
Result:   True
Intersecting values: set()


# II Sposób:

### Omówienie algorytmu

Sortujemy obie tablice, a następnie przeglądamy je liniowo. Jest to wolniejszy algorytm, ale nie wykorzystuje wyszukiwania binarnego.

### Implementacja algorytmu

In [5]:
def are_disjoint(arr1: list, arr2: list) -> bool:
    # Sort both arrays
    quick_sort(arr1)
    quick_sort(arr2)
    
    if len(arr1) < len(arr2):
        shorter = arr1
        longer = arr2
    else:
        shorter = arr2
        longer = arr1
    # Move linearly through both arrays and check if any of the values
    # from the shorter array is in the longer array
    long_idx = 0
    for val in shorter:
        # Move a pointer to the same value candidate
        while long_idx < len(longer) and longer[long_idx] < val:
            long_idx += 1
        if long_idx < len(longer) and longer[long_idx] == val:
            return False
    return True


def quick_sort(arr):
    _quick_sort(arr, 0, len(arr) - 1)
    

def _quick_sort(arr, left_idx, right_idx):
    while left_idx < right_idx:
        pivot_position = _partition(arr, left_idx, right_idx)
        
        if pivot_position - left_idx < right_idx - pivot_position:
            _quick_sort(arr, left_idx, pivot_position)
            left_idx = pivot_position + 1  # I removed a tailing recursion
        else:
            _quick_sort(arr, pivot_position + 1, right_idx)
            right_idx = pivot_position  # I removed a tailing recursion
        
        
def _partition(arr, left_idx, right_idx):
    pivot = arr[left_idx]
    
    # Partition an array into 2 subarrays of elements lower than or
    # equal to a pivot and of elements greater or equal to a pivot 
    # (in this partition algorithm pivot isn't placed on a fixed position 
    # but can be also swapped like all the remaining values)
    i = left_idx - 1
    j = right_idx + 1
    while True:
        i += 1
        while arr[i] < pivot: i += 1
            
        j -= 1
        while arr[j] > pivot: j -= 1
        
        if i < j:
            _swap(arr, i, j)
        else:
            return j  # Return a pivot position after the last swap

    
def _swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]

###### Kilka testów

In [6]:
a = [1, 2, 6, 2, 1, 7, 3]
b = [0, 5, 5, 4, -4, 12]
print(are_disjoint(a, b))

True


In [7]:
a = [1, 2, 6, 2, 1, 7, 3]
b = [0, 5, 5, 4, -4, 12, 2]
print(are_disjoint(a, b))

False


In [8]:
import random

longer_length  = 2**16
shorter_length = 2**2

mul = 100
round_ = 3
arr1 = [round(random.random() * mul * random.choice([-1, 1]), round_) for _ in range(longer_length)]
arr2 = [round(random.random() * mul * random.choice([-1, 1]), round_) for _ in range(shorter_length)]

print(len(arr1), len(arr2))
intersection = set(arr1) & set(arr2)
expected = not bool(intersection)
result = are_disjoint(arr1, arr2)
print('Expected:', expected)
print('Result:  ', result)
print('Intersecting values:', intersection)
for val in intersection:
    print(f'Is {val} in arr1?: {val in arr1}\tIs {val} in arr2?: {val in arr2}')

65536 4
Expected: False
Result:   False
Intersecting values: {23.181}
Is 23.181 in arr1?: True	Is 23.181 in arr2?: True


# III Sposób
### (Nie mam pewności, czy zostałby zaakceptowany)

### Opis algorytmu

Postępujemy podobnie jak w 1. sposobie, ale do sortowania używamy ulepszonej wersji Bucket Sorta (niskie ryzyko najgorszego przyadku, duże ryzyko zajęcia nadmiernej pamięci).

### Implementacja algorytmu

In [9]:
def are_disjoint(arr1: list, arr2: list) -> bool:
    # Check which array has more elements
    if len(arr1) < len(arr2):
        shorter = arr1
        longer = arr2
    else:
        shorter = arr2
        longer = arr1
    # Sort a shorter array
    bucket_sort(shorter)
    # Check using a Binary Search if a shorter array
    # has at least element from the longer array
    for val in longer:
        val_idx = binary_search_first(shorter, val)
        if val_idx >= 0:
            return False
    return True


# Use k as a thershold which idicates when to start using Insertion Sort
def bucket_sort(arr, *, k: 'threshold' = 24):
    # If a bucket is small enough, use an Insertion Sort algorithm to
    # sort this bucket
    if len(arr) <= k:
        insertion_sort(arr)
    else:
        _bucket_sort(arr, k)
        
        
def _bucket_sort(arr, k):
    # Store the maximum and the minimum value of a bucket
    min_val, max_val = minmax(arr)
    # Sort a bucket if only there is more than one unique value
    if min_val != max_val:
        # Make a threshold a bit smaller as a number of elements in each
        # bucket can slightly vary and we don't want to make unnecessary
        # recursive calls.
        m = int(2/3 * k)
        # Create buckets
        buckets_count = len(arr) // m + 1
        buckets = [[] for _ in range(buckets_count)]
        val_interval = (max_val - min_val) / buckets_count
        # Distribute values to the proper buckets
        for val in arr:
            # Calculate the bucket's index depending on how much the 
            # current value is greater than the lowest one
            bucket_idx = int((val - min_val) / val_interval - .5)
            buckets[bucket_idx].append(val)
        # Sort each bucket separately
        for bucket in buckets:
            # Bucket sort all of the buckets again
            bucket_sort(bucket, k=k)
        # Rewrite sorted values from buckets to the inintial array
        i = 0
        for bucket in buckets:
            for val in bucket:
                arr[i] = val
                i += 1
        
        
def minmax(arr):
    global_min = global_max = arr[-1]
    
    for i in range(0, len(arr)-1, 2):
        if arr[i] > arr[i+1]:
            if arr[i] > global_max:   global_max = arr[i]
            if arr[i+1] < global_min: global_min = arr[i+1]
        else:
            if arr[i+1] > global_max: global_max = arr[i+1]
            if arr[i] < global_min:   global_min = arr[i]
    return global_min, global_max


def insertion_sort(arr):
    for i in range(1, len(arr)):
        j = i-1
        temp = arr[i]
        
        while j >= 0 and temp < arr[j]:
            arr[j+1] = arr[j]
            j -= 1
        
        arr[j+1] = temp        


def binary_search_first(arr: 'sorted sequence', el: 'searched element') -> int:
    left_idx = 0
    right_idx = len(arr)-1
    
    while left_idx <= right_idx:
        mid_idx = (left_idx + right_idx) // 2
        if el > arr[mid_idx]:
            left_idx = mid_idx + 1
        else:
            right_idx = mid_idx - 1
            
    return left_idx if left_idx < len(arr) and arr[left_idx] == el else -1

###### Kilka testów

In [10]:
a = [1, 2, 6, 2, 1, 7, 3]
b = [0, 5, 5, 4, -4, 12]
print(are_disjoint(a, b))

True


In [11]:
a = [1, 2, 6, 2, 1, 7, 3]
b = [0, 5, 5, 4, -4, 12, 2]
print(are_disjoint(a, b))

False


Puszczając poniższy test, możemy zauważyć, jak szybko działa sortowanie. Więcej czasu zajmuje wygenerowanie losowych liczb niż działanie tego algorytmu.

In [12]:
import random

longer_length  = 2**20
shorter_length = 2**2

mul = 1000
round_ = 4
arr1 = [round(random.random() * mul * random.choice([-1, 1]), round_) for _ in range(longer_length)]
arr2 = [round(random.random() * mul * random.choice([-1, 1]), round_) for _ in range(shorter_length)]

print(len(arr1), len(arr2))
intersection = set(arr1) & set(arr2)
expected = not bool(intersection)
result = are_disjoint(arr1, arr2)
print('Expected:', expected)
print('Result:  ', result)
print('Intersecting values:', intersection)
for val in intersection:
    print(f'Is {val} in arr1?: {val in arr1}\tIs {val} in arr2?: {val in arr2}')

1048576 4
Expected: True
Result:   True
Intersecting values: set()
